In [4]:
import pandas as pd
import numpy as np

In [6]:
# load in the data from a csv file
data = pd.read_csv("/home/kosimadukwe/supermarket_sales.csv", sep=",")


In [8]:
# peep at the first 5 rows to see the column titles
print(data.head().transpose)

<bound method DataFrame.transpose of     Invoice ID Branch       City Customer type  Gender  \
0  750-67-8428      A     Yangon        Member  Female   
1  226-31-3081      C  Naypyitaw        Normal  Female   
2  631-41-3108      A     Yangon        Normal    Male   
3  123-19-1176      A     Yangon        Member    Male   
4  373-73-7910      A     Yangon        Normal    Male   

             Product line  Unit price  Quantity   Tax 5%     Total  \
0       Health and beauty       74.69         7  26.1415  548.9715   
1  Electronic accessories       15.28         5   3.8200   80.2200   
2      Home and lifestyle       46.33         7  16.2155  340.5255   
3       Health and beauty       58.22         8  23.2880  489.0480   
4       Sports and travel       86.31         7  30.2085  634.3785   

         Date         Time      Payment    cogs  gross margin percentage  \
0  01-05-2019   1:08:00 PM      Ewallet  522.83                 4.761905   
1  03-08-2019  10:29:00 AM         Cash  

In [9]:
# inspect the data types of all the columns to see if its expected
print(data.dtypes)

Invoice ID                  object
Branch                      object
City                        object
Customer type               object
Gender                      object
Product line                object
Unit price                 float64
Quantity                     int64
Tax 5%                     float64
Total                      float64
Date                        object
Time                        object
Payment                     object
cogs                       float64
gross margin percentage    float64
gross income               float64
Rating                     float64
dtype: object


In [10]:
# convert the wrong data types to the correct one
data[['Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Payment']] \
    = data[['Branch', 'City', 'Customer type', 'Gender', 'Product line', 'Payment']].astype("string")

In [11]:
# the Date column column contains different formats 00/00/0000 and 00-00-0000
data['Date'] = pd.to_datetime(data['Date'], dayfirst=True)

In [12]:
# i would like to split  this into specific columns
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day

In [13]:
'''
Which are the Top product_line by Total sales?
'''
# Total Sales = (Unit Price * Quantity) + Tax 5%
# we already have this provided in Total column, so we don't need to calculate it

# to get the top sales, group Total column by Product line and then sort in descending order
data2 = pd.DataFrame(data.groupby('Product line')['Total'].sum())
TopProduct_line_by_Total_sales = data2.sort_values(by='Total', ascending=False)
print("The top product lines by total sales are : ", TopProduct_line_by_Total_sales)

#save to file for PowerBI viz
TopProduct_line_by_Total_sales = TopProduct_line_by_Total_sales.reset_index()
TopProduct_line_by_Total_sales.to_csv("TopProduct_line_by_Total_sales.csv",index=False)

The top product lines by total sales are :                               Total
Product line                      
Food and beverages      56144.8440
Sports and travel       55122.8265
Electronic accessories  54337.5315
Fashion accessories     54305.8950
Home and lifestyle      53861.9130
Health and beauty       49193.7390


In [14]:
'''
Which are the most selling product_line?
'''
# this is determined by the quantity of each product line sold
# Assumption made: The Quantity column refers to Quantity Sold and not Quantity Available/Produced

# to get the top selling product line, group Quantity column by Product line and then sort in descending order
data2 = pd.DataFrame(data.groupby('Product line')['Quantity'].sum())
TopProduct_line_by_QuantitySold = data2.sort_values(by='Quantity', ascending=False)
print("The top product line by quantity sold are : ", TopProduct_line_by_QuantitySold)

#save to file for PowerBI viz
TopProduct_line_by_QuantitySold = TopProduct_line_by_QuantitySold.reset_index()
TopProduct_line_by_QuantitySold.to_csv("TopProduct_line_by_QuantitySold.csv",index=False)


The top product line by quantity sold are :                          Quantity
Product line                    
Electronic accessories       971
Food and beverages           952
Sports and travel            920
Home and lifestyle           911
Fashion accessories          902
Health and beauty            854


In [16]:
'''
Which are the most preferred payment mode?
'''
# count the unique occurrences from the Payment column

Preferred_Payment = pd.DataFrame(data['Payment'].value_counts(sort=True))
print("The Most Preferred Payment Methods : ", Preferred_Payment)
#save to file for PowerBI viz
Preferred_Payment = Preferred_Payment.reset_index()
Preferred_Payment.to_csv("Preferred_Payment_Mode.csv",index=False)


The Most Preferred Payment Methods :               Payment
Ewallet          345
Cash             344
Credit card      311


In [19]:
'''
The most preferred means of payment for Each gender.
'''
# group Preferred_Payment by Gender
# for each gender, redo the above preferred payment process

# subset the dataframe where Gender == Male
Male_subset = data.loc[data['Gender'] == "Male"]
Preferred_Payment1 = pd.DataFrame(Male_subset['Payment'].value_counts(sort=True))
      
#save to file for PowerBI viz
Preferred_Payment1 = Preferred_Payment1.reset_index()
Preferred_Payment1.to_csv("Preferred_Payment_Mode_Male.csv",index=False)
print("The Most Preferred Payment Methods for Male : ", Preferred_Payment1)

# subset the dataframe where Gender == Female
Female_subset = data.loc[data['Gender'] == "Female"]
Preferred_Payment2 = pd.DataFrame(Female_subset['Payment'].value_counts(sort=True))

#save to file for PowerBI viz
Preferred_Payment2 = Preferred_Payment2.reset_index()
Preferred_Payment2.to_csv("Preferred_Payment_Mode_Female.csv",index=False)
print("The Most Preferred Payment Methods for Female : ", Preferred_Payment2)


The Most Preferred Payment Methods for Male :           index  Payment
0      Ewallet      185
1         Cash      166
2  Credit card      148
The Most Preferred Payment Methods for Female :           index  Payment
0         Cash      178
1  Credit card      163
2      Ewallet      160


In [20]:
'''
Which Gender spends the most amount on each product_line?
'''
# for each of the product line show the top spending gender

# subset the dataframe where Gender == Male
Male_subset = data.loc[data['Gender'] == "Male"]
Product_line_by_Total_sales_Males = pd.DataFrame(Male_subset.groupby("Product line")['Total'].sum())
# renaming this so we can merge with the female df
Product_line_by_Total_sales_Males.rename(columns={'Total': 'Male'}, inplace=True)


# subset the dataframe where Gender == Female
Female_subset = data.loc[data['Gender'] == "Female"]
Product_line_by_Total_sales_Females = pd.DataFrame(Female_subset.groupby("Product line")['Total'].sum())
# renaming this so we can merge with the male df
Product_line_by_Total_sales_Females.rename(columns={'Total': 'Female'}, inplace=True)

# merge the male and female aggregated tables
PL_by_TS_Male_Female = Product_line_by_Total_sales_Males.merge(Product_line_by_Total_sales_Females, on='Product line')

# compare the contents of two columns based on multiple conditions and assigned values to a new column

conditions = ((PL_by_TS_Male_Female['Male'] > PL_by_TS_Male_Female['Female']),
              (PL_by_TS_Male_Female['Male'] < PL_by_TS_Male_Female['Female']),
              (PL_by_TS_Male_Female['Male'] == PL_by_TS_Male_Female['Female']))
choices = ['Male', 'Female', 'Both']
PL_by_TS_Male_Female['Top Spending Gender'] = np.select(conditions, choices, default=np.nan)
print('Top Spending Gender for each Product line : ', PL_by_TS_Male_Female)

#save to file for PowerBI viz
PL_by_TS_Male_Female = PL_by_TS_Male_Female.reset_index()
PL_by_TS_Male_Female.to_csv("Top_Spending_Gender_Per_ProductLine.csv",index=False)


Top Spending Gender for each Product line :                                Male      Female Top Spending Gender
Product line                                                      
Electronic accessories  27235.5090  27102.0225                Male
Fashion accessories     23868.4950  30437.4000              Female
Food and beverages      22973.9265  33170.9175              Female
Health and beauty       30632.7525  18560.9865                Male
Home and lifestyle      23825.0355  30036.8775              Female
Sports and travel       26548.1055  28574.7210              Female


In [21]:
'''
Which Gender spends the highest amount in total?
'''
print("Total_Amount_Spent_By_Males", Male_subset['Total'].sum())
print("Total_Amount_Spent_By_Females", Female_subset['Total'].sum())

if Male_subset['Total'].sum() > Female_subset['Total'].sum():
    print("Males spent the highest total amount : {}".format(Male_subset['Total'].sum()))
else:
    print("Females spent the highest total amount: {}".format(Female_subset['Total'].sum()))



Total_Amount_Spent_By_Males 155083.824
Total_Amount_Spent_By_Females 167882.925
Females spent the highest total amount: 167882.925


In [22]:
'''
Top rated product_line.
'''
# aggregate the product line via their rating using groupby
TopRated = pd.DataFrame(data.groupby('Product line')['Rating'].sum())

# round up to a whole number
TopRated['Rating'] = TopRated['Rating'].round(decimals=0)

# sort
TopRated = TopRated.sort_values(by='Rating', ascending=False)

# convert the index to a column
TopRated = TopRated.reset_index()

# show result
print('Top rated product is {} with {} total ratings: '.format(TopRated['Product line'][0], TopRated['Rating'][0]))

#save to file for PowerBI viz
TopRated.to_csv("Top_Rated.csv",index=False)

Top rated product is Fashion accessories with 1251.0 total ratings: 
